# 2.4.4. Оценка семантической близости вопросов и ответов с использованием модели Elmo сервиса RusVectōrēs.  

Видеоуроки:  
2.4.1. Векторное представление единиц текста.  
2.4.2. Оценка близости двух текстов.  
2.4.3. Модель word2vec сервиса RusVectōrēs.  

Дополнительные материалы:  
2.4.4. Скринкаст "Оценка семантической близости вопросов и ответов с использованием модели FastText сервиса RusVectōrēs".  
**2.4.5. Скринкаст "Оценка семантической близости вопросов и ответов с использованием модели Elmo сервиса RusVectōrēs".**  

В предыдущем уроке мы рассмотрели использование предобученной модели FastText сервиса RusVectores. FastText обучается на побуквенных н-граммах, позволяя таким образом обрабатывать слова, которых не было в обучающих примерах.  
У модели FastText (как и Word2vec) есть один недостаток - так как модели обучаются выдавать вектор слова в соответствии с контекстом, в которых оно встречалось, то мы получаем некий усредненный контекст слова. Например в 2 текстах "гриф гитары" и "гриф и имеет острый клюв", "смыслом" слово гриф будет что-то среднее между гитарой и птицей. Эти модели не учитывают различия в контексте. И для слова гриф, вне зависимоси от конекста, модель всегда будет выдавать одинаковый вектор.  
Решить эту проблему пытались как минимум с 2015 года. Сейчас господствующая парадигма в NLP такова: необходимо учитывать контекст не только при обучении моделей, но и при генерации векторов в практических приложениях. Ведь мы, люди, решаем, в каком именно значении употреблено слово, на основании других слов вокруг него.  
То есть, на вход модели должно поступать не одно изолированное слово, а последовательность слов (например, предложение). Модель обрабатывает его целиком и генерирует для каждого слова его вектор, учитывая текущий контекст. Таким образом, в предложениях "гриф гитары" и "гриф имеет острый клюв" для слова "гриф" будут сгенерированы два разных вектора.  

Подобные модели называются "контекстуализированными" (contextualized embeddings) и являются сейчас новым стандартом в автоматической обработке текста. На сайте проекта [RusVectōrēs](https://rusvectores.org/ru/models/) вы можете скачать модели обученные при помощи алгоритма Embeddings from Language Models (ELMo).  
Описание алгоритма - в статье Мэттью Петерса и других ["Deep contextualized word representations"](https://aclweb.org/anthology/N18-1202).  

Для работы с моделями ELMo написана библиотека (simple elmo)[https://github.com/ltgoslo/simple_elmo]

In [25]:
# импортируем все необходимые зависимости.
import warnings
import logging
from typing import Dict, List
import zipfile
import wget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn import metrics
import tensorflow as tf
import simple_elmo


# Общие настройки.
plt.style.use('ggplot')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
warnings.filterwarnings('ignore')

В папке `'./data/volgatech_faq'` представлен датасет:  
Описание файлов данных:  
    `questions.csv` - вопросы пользователей;  
    `answers.csv` - ответы на вопросы;  
    `pos_relations.csv` - правильные ответы на вопросы (пары вопрос==ответ);  
    `neg_relations.csv` - неправильные (неподходящие) ответы.  

In [2]:
!ls ./data/volgatech_faq

answers.csv       neg_relations.csv questions.csv
data.zip          pos_relations.csv


In [3]:
df_questions = pd.read_csv('./data/volgatech_faq/questions.csv', sep=';', names=['id', 'text'])
df_questions.head()

,id,text
0,3647,размерность векторного пространства
1,3644,расстояние
2,3643,привет
3,3631,что такое подпространство
4,3630,матрица перехода к новому базису


In [4]:
# Посчитаем количество уникальных текстов
df_questions['text'].nunique()

92

In [5]:
df_answers = pd.read_csv('./data/volgatech_faq/answers.csv', sep=';', names=['id', 'text'])
df_answers.head()

,id,text
0,339,Размерностью векторного пространства (ВП) назы...
1,358,С Евклидовым расстоянием мы с вами хорошо знак...
2,383,Данный форум не предназначен для решения орган...
3,384,Данный форум не предназначен для решения орган...
4,374,Опр. Подпространством линейного пространства V...


In [6]:
# Посчитаем количество уникальных текстов
df_answers['text'].nunique()

32

In [7]:
# Для удобства работы далее нам нужно отсортировать значения таблиц вопросов и ответов по колонке 'id'

df_answers = df_answers.sort_values(by=['id'], axis=0, ignore_index=True)
df_questions = df_questions.sort_values(by=['id'], axis=0, ignore_index=True)

In [8]:
df_answers.head()

,id,text
0,327,"Векторные пространства, в которых задано скаля..."
1,329,"А сейчас отметим следующее, что, задав скалярн..."
2,331,"Это поле рациональных чисел, с которыми мы фак..."
3,334,Давайте введем понятие абстрактного векторного...
4,336,Кроме геометрических векторов - направленных о...


In [9]:
df_positive = pd.read_csv('./data/volgatech_faq/pos_relations.csv', sep=';', names=['question', 'answer'])
df_positive.head()

,question,answer
0,3647,339
1,3644,358
2,3643,"383,384"
3,3631,374
4,3630,405


В данных 1 вопросу может соотвествовать несколько ответов.  
Изменим данные - так чтобы 1 вопросу соответствовал 1 ответ для упрощения нашего учебного примера, оставим только первый ответ.  

In [10]:
df_positive['answer_1'] = df_positive['answer'].apply(lambda t: [int(a.strip()) for a in t.split(',')][0])

In [11]:
df_positive.head()

,question,answer,answer_1
0,3647,339,339
1,3644,358,358
2,3643,"383,384",383
3,3631,374,374
4,3630,405,405


Скачаем модель, обученную на НКРЯ и Википедии (По сравнению с другими моделями на сайте проекта, тексты Википедии наиболее близки по теме к текстам нашего датасета).  

In [12]:
MODEL_URL = 'http://vectors.nlpl.eu/repository/20/195.zip'

model_arch_file = MODEL_URL.split('/')[-1]
model_path = model_arch_file.split('.')[0]

In [13]:
# Скачивание модели.
# размер файла - 200 Mb.
# Не выполняйте этот код повторно, если модель уже скачана и распакована.

# _ = wget.download(MODEL_URL)
# print(f'extract {model_arch_file} to path: {model_path}')
# with zipfile.ZipFile(model_arch_file, 'r') as archive:
#     zipfile.ZipFile.extractall(archive, model_path)

In [14]:
graph = tf.Graph()

with graph.as_default() as elmo_graph:
    elmo_model = simple_elmo.ElmoModel()
    elmo_model.load(model_path)

with elmo_graph.as_default() as current_graph:
    tf_session = tf.compat.v1.Session(graph=elmo_graph)
    with tf_session.as_default() as sess:
        elmo_model.elmo_sentence_input = simple_elmo.elmo.weight_layers("input", elmo_model.sentence_embeddings_op)
        sess.run(tf.compat.v1.global_variables_initializer())

2022-02-12 19:51:04,146 : INFO : Loading model from 195...
2022-02-12 19:51:04,149 : INFO : We will cache the vocabulary of 100 tokens.
2022-02-12 19:51:05.776497: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-02-12 19:51:05.804423: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa0cf540b50 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-12 19:51:05.804440: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [15]:
def get_text_vectors(texts: List[str]):
    """Получить вектор текста."""

    return elmo_model.get_elmo_vector_average(texts, session=tf_session)

t = df_questions['text'].values[0]
v = get_text_vectors([t])[0]
print('text:', t)
print('размер вектора:', v.size)
print('vector:', list(v[:3]) + ['...'] + list(v[-3:]))

2022-02-12 19:51:15,667 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 19:51:16,947 : INFO : Warming up finished.
2022-02-12 19:51:16,948 : INFO : Texts in the current batch: 1


text: что такое евклидово пространство
размер вектора: 1024
vector: [0.0545633907652542, -0.01873746147080428, -0.08078931645565482, '...', 0.08553287328671184, -0.013187009387740055, 0.01673472377782036]


Попробуем самый простой подход - посчитаем косинусное расстояние между парами вопрос/ответ.  

Чтобы не считать вектора текстов каждый раз - сохраним их в датасете.

In [16]:
df_answers['text_v'] = df_answers['text'].apply(lambda v: get_text_vectors([v])[0])
df_questions['text_v'] = df_questions['text'].apply(lambda v: get_text_vectors([v])[0])

2022-02-12 19:51:20,517 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 19:51:22,316 : INFO : Warming up finished.
2022-02-12 19:51:22,318 : INFO : Texts in the current batch: 1
2022-02-12 19:51:24,291 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 19:51:26,336 : INFO : Warming up finished.
2022-02-12 19:51:26,339 : INFO : Texts in the current batch: 1
2022-02-12 19:51:28,346 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 19:51:30,905 : INFO : Warming up finished.
2022-02-12 19:51:30,909 : INFO : Texts in the current batch: 1
2022-02-12 19:51:33,483 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 19:51:39,779 : INFO : Warming up finished.
2022-02-12 19:51:39,787 : INFO : Texts in the current batch: 1
2022-02-12 19:51:46,455 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 19:51:52,552 : INFO : Warming up finished.
2022-02-12 19:51:52,561 : INFO : Texts in the current batch: 1
2022-02-12 19:51:58,728 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 19

Для того чтобы оценить качество алгоритма, нам нужна метрика качества.  
Применим `top_k_accuracy_score` из пакета `scikit-learn`.  

In [27]:
y_labels = df_answers['id'].values

y_true = [df_positive[df_positive['question'] == q_id]['answer_1'].values[0]
          for q_id in df_questions['id'].values]

y_score = cosine_similarity(list(df_questions['text_v']), list(df_answers['text_v']))

metrics.top_k_accuracy_score(y_true=y_true, y_score=y_score, k=5, labels=y_labels)



0.21390374331550802

Для примера, результат, полученный на модели FastText в предыдущей теме этого курса (2.4.4. Скринкаст "Оценка семантической близости вопросов и ответов с использованием модели FastText сервиса RusVectōrēs"), был равен 0.797

И напишем функцию для получения топ-к ответов:


In [28]:
def get_top_k_answers(q: str, k: int = 5) -> Dict[str, float]:
    """
    get_top_k_answers [summary]

    [extended_summary]

    Args:
        q (str): Текст вопроса.
        k (int, optional): Количество ответов. Defaults to 5.

    Returns:
        Dict[str, float]: Результат в формате {текст_ответа: мера_близости}
    """

    q_vect = get_text_vectors([q])
    y_score = cosine_similarity(q_vect, list(df_answers['text_v']))[0]
    result = {df_answers['text'].values[i]: y_score[i]
             for i in np.argsort(y_score)[::-1][:k]}
    return result

In [29]:
questions = [
    df_questions['text'].values[0],
    'в чём смысл жизни?',
]
for q in questions:
    print('-'*80)
    print('Вопрос:', q)
    for answer, score in get_top_k_answers(q=q).items():
        print(answer[:73]+'...', score)

2022-02-12 20:00:40,813 : INFO : Warming up ELMo on 1 sentences...


--------------------------------------------------------------------------------
Вопрос: что такое евклидово пространство


2022-02-12 20:00:41,110 : INFO : Warming up finished.
2022-02-12 20:00:41,112 : INFO : Texts in the current batch: 1
2022-02-12 20:00:41,387 : INFO : Warming up ELMo on 1 sentences...
2022-02-12 20:00:41,540 : INFO : Warming up finished.
2022-02-12 20:00:41,541 : INFO : Texts in the current batch: 1


Мы ввели понятие абстрактного ВП. И уже начали понимать, что под ВП можно... 0.9235582505970406
Векторные пространства, в которых задано скалярное произведение называютс... 0.923190256972104
Мы будем говорить, что линейное пространство конечномерно, если либо оно ... 0.9208890641068799
Размерностью векторного пространства (ВП) называется максимальное кол-во ... 0.9190545034524038
Что опять же интересно – это то, что задав метрику – функцию расстояния, ... 0.9188150796986365
--------------------------------------------------------------------------------
Вопрос: в чём смысл жизни?
Это поле рациональных чисел, с которыми мы фактически и работаем на компь... 0.8385716925544145
Косинус угла между двумя векторами x и y вводится как отношение скалярног... 0.8313879912971116
Данный форум не предназначен для решения организационных вопросов. Вы мож... 0.8301008166134773
Векторные пространства, в которых задано скалярное произведение называютс... 0.8284527508454376
Давайте дадим определение изо

# Выводы 


1. Вектора, предобученные на больших корпусах текстов могут давать неплохой результат "из коробки".  
2. Подбор оптимальной модели векторизации текста (обученной на разных корпусах) может привести к значительно лучшим результатам.  